In [1]:
# https://stackoverflow.com/questions/39299838/how-do-i-import-module-in-jupyter-notebook-directory-into-notebooks-in-lower-dir
# https://stackoverflow.com/questions/68572852/import-local-modules-in-jupyter-notebook
import os
import sys
# os.path.split(os.getcwd())[0]
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/models")
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools")
for nb_dir in sys.path:
    print(nb_dir)
    # sys.path.append(nb_dir)


/gpfs/home/dhuang/thesis/WavCaps
/home/dhuang/.conda/envs/fs/lib/python310.zip
/home/dhuang/.conda/envs/fs/lib/python3.10
/home/dhuang/.conda/envs/fs/lib/python3.10/lib-dynload

/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/huggingface_hub-0.14.1-py3.8.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/sentencepiece-0.1.95-py3.10-linux-x86_64.egg
/gpfs/home/dhuang/thesis/WavCaps/retrieval/models
/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools


In [22]:
import torch
import torch.nn as nn
from models.audio_encoder import AudioEncoder
from models.text_encoder import TextEncoder
import torch.nn.functional as F
import copy
from tools.losses import AudioTextContrastiveLoss, NTXent
from tools.utils import remove_grad
import ruamel.yaml as yaml
import librosa
import random
import numpy as np


In [15]:
config_file_path = './settings/baseline.yaml'

with open(config_file_path, "r") as f:
        config = yaml.safe_load(f)

In [26]:
# Load audio signal file
wav_file_path = '../../dac/audio_samples/at2_cvt.wav'
waveform, _ = librosa.load(wav_file_path, sr=config["audio_args"]["sr"], mono=True)
print('waveform shape before crop: ', waveform.shape)
if config["audio_args"]["max_length"] != 0:
            # if audio length is longer than max_length, we random crop it
            max_length = config["audio_args"]["max_length"] * config["audio_args"]["sr"]
            if waveform.shape[-1] > max_length:
                max_start = waveform.shape[-1] - max_length
                start = random.randint(0, max_start)
                waveform = waveform[start: start + max_length]
                
print('waveform shape: ', waveform.shape)
waveform_tensor = torch.tensor(waveform[None, :])
print('waveform_tensor shape: ', waveform_tensor.shape)


waveform shape before crop:  (480000,)
waveform shape:  (320000,)
waveform_tensor shape:  torch.Size([1, 320000])


In [18]:
audio_encoder = AudioEncoder(config)
# settings for projection layers
embed_size = config["embed_size"]
audio_width = audio_encoder.audio_width

In [ ]:
# audio_encoded = audio_encoder.audio_enc(inputs)

In [29]:
with torch.no_grad():
    # audio_encoded = audio_encoder.audio_enc(np.ndarray([[0], waveform]) )
    audio_encoded = audio_encoder.audio_enc(waveform_tensor)
    # audio_encoded = audio_encoder.audio_enc(torch.randn(1, 1, 64, 1024))
    # audio_feats = audio_encoder(waveform)
    # audio_embeds = F.normalize(audio_proj(audio_feats), dim=-1)

print('audio_encoded.shape: ', audio_encoded.shape)

audio_encoded.shape:  torch.Size([1, 768])
